<a href="https://colab.research.google.com/github/arssite/Datalysis/blob/main/VGG16_DogVsCat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**VGG-16 | CNN model**
ImageNet Large Scale Visual Recognition Challenge (ILSVRC) is an annual computer vision competition. Each year, teams compete on two tasks. The first is to detect objects within an image coming from 200 classes, which is called object localization. The second is to classify images, each labeled with one of 1000 categories, which is called image classification. VGG 16 was proposed by Karen Simonyan and Andrew Zisserman of the Visual Geometry Group Lab of Oxford University in 2014 in the paper “VERY DEEP CONVOLUTIONAL NETWORKS FOR LARGE-SCALE IMAGE RECOGNITION”. This model won 1st  and 2nd place in the above categories in the 2014 ILSVRC challenge.

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d salader/dogs-vs-cats

100% 1.06G/1.06G [00:48<00:00, 24.4MB/s]
100% 1.06G/1.06G [00:48<00:00, 23.4MB/s]


In [3]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [6]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [7]:
conv_base = VGG16(
    weights = 'imagenet',
    include_top = False, # remove the dense
    input_shape = (150,150,3) # reshaping
)

58889256/58889256 [==============================] - 3s 0us/step


In [8]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [9]:
model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 16812353 (64.13 MB)
Trainable params: 2097665 (8.00 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [16]:
conv_base.trainable = False

In [20]:
# Use when we are not doing data augmentation
train_ds = keras.utils.image_dataset_from_directory(
directory = '/content/train',
                                                     labels='inferred',
                                                     label_mode='int',
                                                     batch_size=32,

     image_size = (150,150)
 )
validation_ds = keras.utils.image_dataset_from_directory(
     directory = '/content/test',
     labels='inferred',
                                                     label_mode='int',
                                                     batch_size=32,
     image_size = (150,150)

     )

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [21]:
def process(image,label):
     image = tensorflow.cast(image/255.,tensorflow.float32)
     return image,label
train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [22]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds,epochs=10,validation_data=validation_ds)

Epoch 1/10
625/625 [==============================] - 73s 104ms/step - loss: 0.2828 - accuracy: 0.8849 - val_loss: 0.2077 - val_accuracy: 0.9106
Epoch 2/10
 23/625 [>.............................] - ETA: 41s - loss: 0.1910 - accuracy: 0.9226

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'],color='red',label='train')
plt.plot(history.history['val_accuracy'],color='blue',label='validation')

In [23]:
# history = model.fit(train_ds,epochs=10,validation_data=validation_ds)
history = model.fit_generator(
    train_generator,
    epochs = 10,
    validation_data = validation_generator
)

NameError: name 'train_generator' is not defined

In [ ]:
print('accuracy: ',history.history['accuracy'][9])
print('val_accuracy: ',history.history['val_accuracy'][9])

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])